In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

from tqdm.notebook import tqdm

In [8]:
for s in tqdm(range(48)):
    train = pd.read_csv('train.csv')
    train['retro_date'] = pd.to_datetime(train['retro_date'])

    test = pd.read_csv('test.csv')
    test['retro_date'] = pd.to_datetime(test['retro_date'])

    sub = pd.read_csv('sample_submission.csv', index_col=0)

    cat_train = ['cc_2', 'cc_4', 'cc_6']
    lst=[]
    for i in cat_train:
        for j in train[i]:
            lst.append(j)

    le = LabelEncoder()
    le.fit(lst)
    for col in cat_train:
        train[col] = le.transform(train[col])
    
    X = train.drop(['successful_utilization'], axis = 1)
    y = train.successful_utilization.values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

    cat = CatBoostClassifier(task_type='GPU')
    cat.fit(X, y, verbose=0)

    model = cat
    for col in cat_train:
        test[col] = le.transform(test[col])
    
    test_0 = test.copy()
    test_0['treatment'] = 0
    test_1 = test.copy()
    test_1['treatment'] = 1

    pred_0 = model.predict_proba(test_0)[:,1]
    pred_1 = model.predict_proba(test_1)[:,1]
    uplift = pred_1 - pred_0

    sub['successful_utilization'] = uplift
    sub.to_csv('subs1/sample'+str(s)+'.csv')

  0%|          | 0/48 [00:00<?, ?it/s]